# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Week 5: Transformer Architecture</font>

# <font color="#003660">Notebook 2: Text Classification with Transformers</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>
<center>
<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... will know the difference between the feature extraction and fine.-tuning approach for text classification with Transformers, <br>
        ... will know how to train a feature extraction model for text classification, <br>
        ... will know how to train a fine-tuned model for text classification, <br>,
        ... will get to know the main libraries of the Hugging Face ecosystem (i.e., Datasets, Tokenizers, Transformers).
    </font>
</div>
</center>
</p>

The following content is heavily inspired by the following excellent sources:


*   Tunstall et al. (2021): Natural Language Processing with Transformers. O'Reilly. https://www.oreilly.com/library/view/natural-language-processing/9781098103231/
*   Hugging Face (2021): Transformer Models - Hugging Face Course. https://huggingface.co/course/



# Import Packages

As always, we first need to load a number of required Python packages:
- `pandas` provides high-performance, easy-to-use data structures and data analysis tools.
- `numpy` is a library adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
- `sklearn` is a free software machine learning library for the Python programming language.
- `transformers` provides general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages.
- `datasets` is an API for datasets from the makers of transformers.
- `torch` is an open source machine learning library used for applications such as computer vision and natural language processing, primarily developed by Facebook's AI Research lab.

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install accelerate -U

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

# Overview

<center><br><img width=400 src="https://upload.wikimedia.org/wikipedia/en/f/f1/Bert_and_Ernie.JPG"/><br></center>



In this notebook we will use a famous Transformer model called **BERT**, short for Bidirectional Encoder Representations from Transformers.

We will use the three core libraries from the Hugging Face ecosystem: **Datasets**, **Tokenizers**, and **Transformers**. These libraries will allow us to quickly go from raw text to a fine-tuned model that can be used for predictions on new texts.

# Load and Prepare Dataset

In the next code chunks we will use the **Datasets** library to retrieve a dataset from Hugging Face's datasets hub. This library is designed to load and process large datasets efficiently and share them with the community.

Let's start by getting a list of datasets from the hub.

In [ ]:
datasets = list_datasets()
print(f"There are {len(datasets)} datasets currently available on the Hub.")
print(f"The first 10 are: {datasets[:10]}")

Load the "emotions" dataset, a dataset of English Twitter messages labelled with six basic emotions: anger, fear, joy, love, sadness, and surprise. More: https://huggingface.co/datasets/emotion

In [ ]:
emotions = load_dataset("emotion")

Explore the structure of the dataset.

In [ ]:
emotions

In [ ]:
emotions["train"].features

Show the first Tweet and its label.

In [ ]:
emotions["train"][0]

Get the name of the first label.

In [ ]:
emotions["train"].features["label"].int2str(emotions["train"][0]["label"])

Store a list of all label names for later.

In [ ]:
labels = emotions["train"].features["label"].names
labels

# Tokenize Texts

Like other neural networks or machine learning models, Transformer models cannot process raw strings as input; instead they assume the text has been tokenized and turned into numerical vectors.

Most Transformers use a subword **tokenizer**. The idea behind subword tokenization is a meet-in-the-middle between character and word tokenization. On one hand we want to use characters since they allow the model to deal with rare character combinations and misspellings. On the other hand, we want to keep frequent words and word parts as unique entities.

There are many different subword tokenization strategies. Using the right tokenizer for a given pretrained model is crucial for getting sensible results. The Transformers library provides convenient tools (e.g., AutoTokenizer, from_pretrained()) to load both objects (model and tokenizer) from the Hugging Face model hub.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer.model_max_length

In [ ]:
encoded_str = tokenizer.encode("this is a complicatedtest")
encoded_str

Show special tokens used by the tokenizer. BERT uses the [MASK] token for the primary objective of masked language modeling and the [CLS] and [SEP] tokens for the secondary pretraining objective of predicting if two sentences are consecutive.

Below, we can observe two things. First, the [CLS] and [SEP] tokens have been added automatically to the start and end of the sequence. Second, the long word `complicatedtest` has been split into two tokens. The ## prefix in ##test signifies that the preceding string is not a whitespace and that it should be merged with the previous token.

In [ ]:
for token in encoded_str:
    print(token, tokenizer.decode([token]))

# BERT for Text Classification

BERT has been pretrained to predict masked words and next sentences in texts. Hence, we can’t use the model as-is for text classification and have to modify its architecture slightly.

The figure below illustrates the general architecture of a BERT text classification model.

<center><br><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/bert.png"/><br></center>

First, the text is tokenized and represented as one-hot vectors whose dimension is the size of the tokenizer vocabulary (not shown in the diagram). Next, these token encodings are embedded in lower dimensions and passed through stacks of encoder layers to yield a hidden state for each input token. During pretraining, the hidden states are used for the task of language modeling. For the classification task, we replace the language modeling layer with a classification model.

We have two options to train the classification model:

* Feature extraction: We use BERT in inference mode take the hidden states as features to train an "external" classifier on them.

* Fine-tuning: We add a classification head to the model and train the whole model end-to-end, which also updates the parameters of the pretrained BERT model.

# The Feature Extraction Approach

The figure below illustrates the idea behind the feature extraction approach. To use a Transformer as a feature extractor we use BERT just for inference and use the generated hidden states as features for a downstream classifier (e.g., logistic regression, random forest). The advantage of this approach is that we can quickly train a shallow model. This method is especially convenient if GPUs are unavailable since the hidden states can be computed relatively fast on a CPU.

<center><br><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/feature_extraction.png"/><br></center>


See also Jay Alammar's blog post for using the feature-extraction approach for classification: https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

## Model: Load and Inference

Check if GPU is available.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

(Down)Load a pretrained model and move it to the GPU. The AutoModel class initalizes an input encoder that translates the one-hot vectors to embeddings with positional encodings and feeds them through the encoder stack to return the hidden states. Note that the language model head that normally takes the hidden states and decodes them to the masked token prediction is excluded since it is only needed for pretraining.

In [ ]:
model_name = "distilbert-base-uncased"

In [ ]:
model = AutoModel.from_pretrained(model_name).to(device)

Test the model with a single short document.

In [ ]:
text = "this is a test"
text_tensor = tokenizer.encode(text, return_tensors="pt").to(device)

The tokenized text is a tensor with the dimensionality [1, 6] (number of documents, length of the documents). 

In [ ]:
text_tensor.shape

The values of the tensor are integers, which are the indices of the tokens in the vocabulary (101 and 102 are special tokens that have been added during tokenization).

In [ ]:
text_tensor

Let's feed the tensor through the model to get the hidden states.

In [ ]:
output = model(text_tensor)

In [ ]:
output

The output are the hidden states of the last encoder layer for each token. Hence, they have the dimensionality [1, 6, 768] (number of documents, length of the documents, dimension of the hidden states). They are the raw contextual embeddings generated by BERT.

In [ ]:
output.last_hidden_state.shape

In [ ]:
output.last_hidden_state

Let’s repeat the process for the whole dataset. First, we write a simple function that will tokenize our texts. The padding=True parameter will pad the examples with zeroes to the longest one in a batch, and truncation=True will truncate the examples to the model’s maximum sequence length.

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

Apply to the first three texts.

In [ ]:
tokenize(emotions["train"][:3])

Repeat with the whole corpus.

In [ ]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

The above function call has added two new features to the dataset: input_ids and the attention mask.

In [ ]:
emotions_encoded["train"].features

With `hidden_states = model(input_ids, attention_mask)` we could now generate the hidden states for each token of each document. For convenience, we define a custom function that takes a batch of tokenized documents, feeds them through the model, and adds hidden_state features to the batch. Instead of using the hidden states of all tokens, in the original BERT paper it was recommended to only use the hidden states of the CLS token as features for classification. An alternative would be to average over the hidden states of all tokens.

In [ ]:
def extract_features_cls(batch):
  # store inputs in separate variables
  input_ids = torch.tensor(batch["input_ids"]).to(device)
  attention_mask = torch.tensor(batch["attention_mask"]).to(device)

  # feed inputs into model and save outputs
  with torch.no_grad():
      last_hidden_state = model(input_ids, attention_mask).last_hidden_state
      last_hidden_state = last_hidden_state.cpu().numpy()

  # extract the hidden states for the CLS token (i.e., the first token)
  cls_lhs = last_hidden_state[:,0,:]

  # return results
  batch["cls_hidden_state"] = cls_lhs
  return batch

In [ ]:
emotions_encoded = emotions_encoded.map(extract_features_cls,
                                        batched=True,
                                        batch_size=16)

Show the dataset again.

In [ ]:
emotions_encoded["train"].features

Check the dimensions of the hidden states.

In [ ]:
len(emotions_encoded["train"]["cls_hidden_state"][0])

The dataset now contains all the information we need to train a classifier on it. We will use the extracted hidden states of the CLS tokens as input features and the labels as targets. We can easily create the corresponding arrays in the well known Scikit-Learn format as follows:

In [ ]:
y_valid = np.array(emotions_encoded["validation"]["label"])

In [ ]:
X_train = np.array(emotions_encoded["train"]["cls_hidden_state"])
X_valid = np.array(emotions_encoded["validation"]["cls_hidden_state"])
y_train = np.array(emotions_encoded["train"]["label"])
y_valid = np.array(emotions_encoded["validation"]["label"])
X_train.shape, X_valid.shape

And now we can train a standard classifier on these data structures.

In [ ]:
clf = LogisticRegression(max_iter=3000)
clf.fit(X_train, y_train)

## Evaluate Model

Make predictions on validation set and evaluate predictive accuracy.

In [ ]:
y_preds = clf.predict(X_valid)

In [ ]:
print(classification_report(y_valid, y_preds, target_names=labels))

# The Fine-tuning Approach

Let’s now fine-tune a Transformer end-to-end. With the fine-tuning approach we do not use the hidden states as fixed features, but instead train them as shown in the figure below. Since we retrain all the parameters, this approach requires more compute than the feature extraction approach and typically requires a GPU.

<center><br><img width=600 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/fine_tuning.png"/><br></center>


## Model: Load Pre-trained Model and Fine-tune

In the following, we will use the Trainer API to simplify the training loop.

We need is a pretrained model like the one we used in the feature-based approach. The only difference is that we use the AutoModelForSequenceClassification model instead of AutoModel. This model has a classification head on top of the model outputs which can be easily trained with the base model.

In [ ]:
model_name = "distilbert-base-uncased"

In [ ]:
model = (AutoModelForSequenceClassification
         .from_pretrained(model_name, num_labels = 6)
         .to(device))

Furthermore, we define a function to monitor some metrics during training.

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

With the dataset and metrics ready we can now instantiate a Trainer class. The main ingredient here is the TrainingArguments class to specify all the parameters of the training run, one of which is the output directory for the model checkpoints.

In [ ]:
batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size

training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=2,
                                  learning_rate=3e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps)

We can now instantiate and fine-tune our model with the Trainer.

In [ ]:
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"])
trainer.train()

## Evaluate Model

Make predictions on validation set and evaluate predictive accuracy.

In [ ]:
preds_output = trainer.predict(emotions_encoded["validation"])

In [ ]:
preds_output

Above, you can see that the model outputs are raw logits, not probabilities. Nonetheless, we can simply take the highest value as the predicted label.

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
print(classification_report(y_valid, y_preds, target_names=labels))

The classification report reveals that the fine-tune approach is performing much better than the feature extraction approach.